In [1]:
# Bringing both PRM and A* agents
# PRM agent will be the firetruck with kinematic constraints trying to reach the obstacles and extinguish them
# A* agent will be the wumpus trying to reach the obstacles and burn them
# Simulate the environment

import numpy as np
import settings
import pygame
import PRM
import time
import Generate_obstacles
import Wumpus_A_star
from scipy.spatial.distance import euclidean







settings.init()
pygame.display.set_caption('PRM')
#Generate obstacles
obstacle_map = Generate_obstacles.generate_obstacles()

#Generate PRM agent
prm_agent = PRM.PRM(obstacle_map,2000,6)
#Generate A* agent
wumpus_agent = Wumpus_A_star.wumpus_A_star(obstacle_map)
#Generate  points
points = prm_agent.generate_random_points()
#Generate graph
prm_agent.generate_road_map()

pygame 2.3.0 (SDL 2.24.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
62


  8%|▊         | 162/2000 [01:05<12:41,  2.41it/s]

deleted


 12%|█▏        | 242/2000 [01:45<19:24,  1.51it/s]  

deleted


...

 66%|██████▋   | 1326/2000 [09:34<03:49,  2.94it/s]

deleted


 74%|███████▍  | 1483/2000 [10:30<02:36,  3.30it/s]

deleted


100%|█████████▉| 1996/2000 [14:07<00:01,  2.35it/s]


In [2]:
def find_obstacle_to_burn(obstacle_to_burn,points):
    #Find the point nearest to the obstacle
    obstacle_select = np.random.choice(obstacle_to_burn)
    dist = np.float32('inf')
    goal = None
    for point in points:
        if euclidean((point.vertex[0],point.vertex[1]),obstacle_select.get_center()) < dist:
            dist = euclidean((point.vertex[0],point.vertex[1]),obstacle_select.get_center())
            goal = point

    return goal, obstacle_select

def find_obstacle_to_save(obstacles_to_save,points):
    print(len(obstacles_to_save))
    #Find the point nearest to the obstacle
    obstacle_select = np.random.choice(obstacles_to_save)
    dist = np.float32('inf')
    goal = None
    for point in points:
        if euclidean((point.vertex[0],point.vertex[1]),obstacle_select.get_center()) < dist:
            dist = euclidean((point.vertex[0],point.vertex[1]),obstacle_select.get_center())
            goal = point

    return goal, obstacle_select

In [4]:
settings.screen.fill((255, 255, 255))
for tetromino in obstacle_map:
    tetromino.state = 0
    tetromino.time_left_to_live = np.float32('inf')
    tetromino.update(0.1)
    tetromino.draw(settings.screen)
unburnt_obstacles = obstacle_map.copy()
burning_obstacles = []
burnt_obstacles = []
goal,obstacle_to_burn = find_obstacle_to_burn(unburnt_obstacles,points)
start,_ = find_obstacle_to_burn(unburnt_obstacles,points)
#Generate path
#truck_path,_ = prm_agent.generate_road_from_graph(start_point = start, goal_node=goal)
truck_path = []
wumpus_path = wumpus_agent.get_path([start.vertex[0],start.vertex[1]], [goal.vertex[0],goal.vertex[1]])
#Generate controls
for tetromino in obstacle_map:
    tetromino.draw(settings.screen)

# for local_points in truck_path:
#     pygame.draw.circle(settings.screen, (255,0,255), (int(local_points[0]*settings.pixels_per_meter), int(local_points[1]*settings.pixels_per_meter)), 2)

for i in range(len(wumpus_path)-1):
    pygame.draw.line(settings.screen, (0,255,0), (wumpus_path[i][0]*settings.pixels_per_meter, wumpus_path[i][1]*settings.pixels_per_meter), (wumpus_path[i+1][0]*settings.pixels_per_meter, wumpus_path[i+1][1]*settings.pixels_per_meter), 4)

pygame.display.update()

wumpus_goal_reached = True
truck_goal_reached = True
print(start)
start_wumpus = goal
start_truck = start
obstacle_to_burn.update(0.1,'burn')
burning_obstacles.append(obstacle_to_burn)
unburnt_obstacles.remove(obstacle_to_burn)

num_saved = 0
num_burned = 0
wumpus_wait_counter  = 0
current_wumpus_position = [start_wumpus.vertex[0],start_wumpus.vertex[1]]
current_truck_position = [start_truck.vertex[0],start_truck.vertex[1],start_truck.vertex[2]]
print('Starting simulation', len(burning_obstacles))
obstacle_to_save_updated = False
obstacle_to_burn_updated = False

for iteration in range(36000):

    settings.screen.fill((255, 255, 255))
    
    for tetromino in obstacle_map:
        tetromino.draw(settings.screen)

    if truck_goal_reached:
        if(len(burning_obstacles) != 0):
            goal_truck,obstacle_to_save = find_obstacle_to_save(burning_obstacles,points)
            print('start truck',start_truck.vertex)
            print('goal truck',goal_truck.vertex)
            truck_path,_ = prm_agent.generate_road_from_graph(start_point = start_truck, goal_node=goal_truck)
            truck_goal_reached = False
        
    
    if wumpus_goal_reached:
        goal_wumpus, obstacle_to_burn = find_obstacle_to_burn(unburnt_obstacles,points)
        wumpus_path = wumpus_agent.get_path([start_wumpus.vertex[0],start_wumpus.vertex[1]], [goal_wumpus.vertex[0],goal_wumpus.vertex[1]])
        wumpus_goal_reached = False
    
    if wumpus_wait_counter == 1:
        wumpus_wait_counter = 0
        if len(wumpus_path) > 0:
            wumpus_agent.move(wumpus_path[0])
            current_wumpus_position = wumpus_path[0]
            local_points = wumpus_path[0]
            pygame.draw.circle(settings.screen, (0,255,0), (int(local_points[0]*settings.pixels_per_meter), int(local_points[1]*settings.pixels_per_meter)), 5)
            wumpus_path.pop(0)
    else:
        wumpus_wait_counter += 1
        wumpus_agent.move(current_wumpus_position)
        pygame.draw.circle(settings.screen, (0,255,0), (int(current_wumpus_position[0]*settings.pixels_per_meter), int(current_wumpus_position[1]*settings.pixels_per_meter)), 5)
    
    if len(truck_path) > 0:
        prm_agent.move(truck_path[0])
        current_truck_position = truck_path[0]
        local_points = truck_path[0]
        pygame.draw.circle(settings.screen, (255,0,255), (int(local_points[0]*settings.pixels_per_meter), int(local_points[1]*settings.pixels_per_meter)), 5)
        truck_path.pop(0)

        

    if len(wumpus_path) == 0:
        wumpus_goal_reached = True
        start_wumpus = goal_wumpus
        obstacle_to_burn.update(0.1,'burn')
        obstacle_to_burn_updated = True
        burning_obstacles.append(obstacle_to_burn)
        if len(unburnt_obstacles) != 0:
            unburnt_obstacles.remove(obstacle_to_burn)

    
    
    
    if len(truck_path) == 0 and not truck_goal_reached:
        truck_goal_reached = True
        start_truck = goal_truck
        obstacle_to_save.update(0.1,'save')
        obstacle_to_save_updated = True
        if obstacle_to_save.get_status() == 0:
            num_saved +=1
            unburnt_obstacles.append(obstacle_to_save)
            burning_obstacles.remove(obstacle_to_save)

    elif len(truck_path) == 0 and truck_goal_reached:
        prm_agent.move(current_truck_position)

    for obstacle in obstacle_map:
        if obstacle.status == 2:
            burnt_obstacles.append(obstacle)
            print("check",len(burning_obstacles))
            if obstacle in burning_obstacles:
                burning_obstacles.remove(obstacle)

    for tetromino in obstacle_map:
        if tetromino == obstacle_to_burn and obstacle_to_burn_updated:
            obstacle_to_burn_updated = False
            continue
        elif tetromino == obstacle_to_save and obstacle_to_save_updated:
            obstacle_to_save_updated = False
            continue
        tetromino.update(0.1)

    if len(burning_obstacles) == 0 and len(unburnt_obstacles) == 0:
        break

            



    pygame.display.update()
    pygame.time.wait(10)


for obstacle in obstacle_map:
    if obstacle.status == 2:
        num_burned +=1
    


print('Number of obstacles saved',num_saved/len(obstacle_map))
print('Number of obstacles burned',num_burned/len(obstacle_map))
        
    




Starting simulation 1
1


TypeError: cannot unpack non-iterable NoneType object